In [5]:
def generateInputWordFile(filename='/local-scratch/asa224/input'):
    """
    The function generates the word file, similar to the count_1w.txt file provided by Prof. Anoop
    
    The output of the file can be used to parse the characters, and is an input to the 
    generateTupleList() function as well. 
    
    INPUT: Input to this function is the training text file. 
    
    """
    with(open(filename, 'rb')) as f:
        word_file_1M = open('/local-scratch/input_word_file', 'wb')
        for line in f:
            line = unicode(line, 'utf-8')
            line = line.replace('\n', '')
            for word in line:
                word_file_1M.write(word.encode('utf-8') + '\t'.encode('utf-8') + str(0).encode('utf-8') +\
                                           '\n'.encode('utf-8'))
        f.close()
    word_file_1M.close()
    
    
def generateTupleList(filename='/local-scratch/asa224/word_file_1M'):
    """
    This function is similar to the above function in the sense that it assigns labels to each
    character in the training set. The function returns a list of tuples, in which each tuple
    contains a single character and its corresponding label. 
    
    INPUT: Input to this function is a WORD FILE generated by generateWordFile(filename) function. 
    
    Use this function in conjunction with nGramSequenceGenerator(labelledlist, n) to create a
    training set with constant sequence size, which does not require paddings. 
    """
    with(open(filename, 'rb')) as f:
        label = []
        for line in f:
            word, count = line.split('\t')

            # making sure the parsing is going fine
            assert int(count) == 0

            word = unicode(word, 'utf-8')
            if len(word) == 1:
                label.append((word[0], 3))
            else:
                for i, character in enumerate(word):
                    if i == 0: # this is the first letter
                        label.append((character, 0))
                    elif i == (len(word) - 1): # this is the last letter
                        label.append((character, 2))
                    else: # this is somewhere in the middle
                        label.append((character, 1))

        f.close()
        return label
    
def nGramSequenceGenerator(labelledlist, n):
    """
    Takes as input the label list of tuples generated by the code above. 
    The function generates sequence of size "n" from the given list. 
    """
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist

In [4]:
generateInputWordFile(filename='/local-scratch/asa224/input')

In [6]:
tuples = generateTupleList(filename='/local-scratch/input_word_file')

In [8]:
final_input = nGramSequenceGenerator(tuples, n=10)

In [36]:
final_input

[[(u'\u6cd5', 3),
  (u'\u6b63', 3),
  (u'\u7814', 3),
  (u'\u7a76', 3),
  (u'\u4ece', 3),
  (u'\u6ce2', 3),
  (u'\u9ed1', 3),
  (u'\u64a4', 3),
  (u'\u519b', 3),
  (u'\u8ba1', 3)],
 [(u'\u5212', 3),
  (u'\u65b0', 3),
  (u'\u534e', 3),
  (u'\u793e', 3),
  (u'\u5df4', 3),
  (u'\u9ece', 3),
  (u'\uff19', 3),
  (u'\u6708', 3),
  (u'\uff11', 3),
  (u'\u65e5', 3)],
 [(u'\u7535', 3),
  (u'\uff08', 3),
  (u'\u8bb0', 3),
  (u'\u8005', 3),
  (u'\u5f20', 3),
  (u'\u6709', 3),
  (u'\u6d69', 3),
  (u'\uff09', 3),
  (u'\u6cd5', 3),
  (u'\u56fd', 3)],
 [(u'\u56fd', 3),
  (u'\u9632', 3),
  (u'\u90e8', 3),
  (u'\u957f', 3),
  (u'\u83b1', 3),
  (u'\u5965', 3),
  (u'\u5854', 3),
  (u'\u5c14', 3),
  (u'\uff11', 3),
  (u'\u65e5', 3)],
 [(u'\u8bf4', 3),
  (u'\uff0c', 3),
  (u'\u6cd5', 3),
  (u'\u56fd', 3),
  (u'\u6b63', 3),
  (u'\u5728', 3),
  (u'\u7814', 3),
  (u'\u7a76', 3),
  (u'\u4ece', 3),
  (u'\u6ce2', 3)],
 [(u'\u9ed1', 3),
  (u'\u64a4', 3),
  (u'\u519b', 3),
  (u'\u7684', 3),
  (u'\u8ba1', 3),
  (u'

Load the dictionaries

In [11]:
import cPickle as pickle
orig_dict = pickle.load( open( "orig_dict.p", "rb" ) )
ret_dict = pickle.load( open( "ret_dict.p", "rb" ) )

In [13]:
import numpy as np

In [14]:
x = [[]]
count = 0
for i in range(0, len(final_input)): # iterate over the whole dataset
    for j in range(0, len(final_input[i])): # iterate over the current sentence
        try:
            x[i].append(orig_dict[final_input[i][j][0]])
        except KeyError:
            x[i].append(np.random.choice(orig_dict.values()))
            count += 1
    x.append([])

[[19, 128, 353, 322, 320, 1300, 1065, 1016, 953, 116],
 [145, 180, 537, 211, 623, 2016, 2825, 217, 2304, 203],
 [339, 99, 551, 222, 612, 3, 1742, 104, 19, 83],
 [83, 633, 188, 120, 2256, 1452, 2045, 1189, 2304, 203],
 [114, 0, 19, 83, 128, 10, 353, 322, 320, 1300]]

4

In [17]:
from keras.models import load_model
model = load_model('/home/asa224/Desktop/students_less_asa224/Test Folder on Less/model_epoch3.h5')

In [32]:
out_file = open('./output_rnn', 'wb')
for seq in x[:-1]:
    pred_labels = model.predict_classes(np.array(seq).reshape(1,len(seq)))
    # get the class label
    
    for num in range(0, len(pred_labels)):
        char = seq[num]
        if pred_labels[0][num] >= 2:
            print('writing {}'.format(ret_dict[char].encode('utf-8')))
            out_file.write(ret_dict[char].encode('utf-8') + ' '.encode('utf-8'))
        else:
            out_file.write(ret_dict[char].encode('utf-8'))
    
#     out_file.write('\n'.encode('utf-8'))
    
out_file.close()

1/1 [==============================] - 0s
writing 法
1/1 [==============================] - 0s
writing 划
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 说
1/1 [==============================] - 0s
writing 尔
1/1 [==============================] - 0s
writing 记
1/1 [==============================] - 0s
writing 话
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 莱
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 到
1/1 [==============================] - 0s
writing 是
1/1 [==============================] - 0s
writing 以
1/1 [==============================] - 0s
writing 队
1/1 [==============================] - 0s
writing 年
1/1 [==============================] - 0s
writing 来
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 盔
1/1 [=======

1/1 [==============================] - 0s
writing 据
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 有
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing 名
1/1 [==============================] - 0s
writing 说
1/1 [==============================] - 0s
writing 而
1/1 [==============================] - 0s
writing 共
1/1 [==============================] - 0s
writing 赛
1/1 [==============================] - 0s
writing 名
1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 他
1/1 [==============================] - 0s
writing 分
1/1 [==============================] - 0s
writing 期
1/1 [==============================] - 0s
writing 肥
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 三
1/1 [=======

1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 几
1/1 [==============================] - 0s
writing 由
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 人
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 体
1/1 [==============================] - 0s
writing 人
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing （
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 智
1/1 [==============================] - 0s
writing 也
1/1 [==============================] - 0s
writing 有
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 的
1/1 [=======

1/1 [==============================] - 0s
writing 摘
1/1 [==============================] - 0s
writing 得
1/1 [==============================] - 0s
writing 个
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 为
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing ）
1/1 [==============================] - 0s
writing 第
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 日
1/1 [==============================] - 0s
writing 多
1/1 [==============================] - 0s
writing 斯
1/1 [==============================] - 0s
writing 手
1/1 [==============================] - 0s
writing 旬
1/1 [==============================] - 0s
writing 场
1/1 [==============================] - 0s
writing 出
1/1 [==============================] - 0s
writing 个
1/1 [==============================] - 0s
writing 三
1/1 [==============================] - 0s
writing 汰
1/1 [=======

1/1 [==============================] - 0s
writing 球
1/1 [==============================] - 0s
writing 行
1/1 [==============================] - 0s
writing 金
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 赛
1/1 [==============================] - 0s
writing 柏
1/1 [==============================] - 0s
writing 者
1/1 [==============================] - 0s
writing 赛
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 名
1/1 [==============================] - 0s
writing 赛
1/1 [==============================] - 0s
writing 牌
1/1 [==============================] - 0s
writing ”
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 赛
1/1 [==============================] - 0s
writing 栏
1/1 [==============================] - 0s
writing 米
1/1 [==============================] - 0s
writing 金
1/1 [==============================] - 0s
writing 了
1/1 [=======

...

1/1 [==============================] - 0s
writing 三
1/1 [==============================] - 0s
writing 瓦
1/1 [==============================] - 0s
writing 前
1/1 [==============================] - 0s
writing 天
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 赵
1/1 [==============================] - 0s
writing 见
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 席
1/1 [==============================] - 0s
writing 浦
1/1 [==============================] - 0s
writing （
1/1 [==============================] - 0s
writing 督
1/1 [==============================] - 0s
writing 月
1/1 [==============================] - 0s
writing 贯
1/1 [==============================] - 0s
writing 将
1/1 [==============================] - 0s
writing 合
1/1 [==============================] - 0s
writing 和
1/1 [==============================] - 0s
writing 再
1/1 [==============================] - 0s
writing 说
1/1 [=======

1/1 [==============================] - 0s
writing 要
1/1 [==============================] - 0s
writing 界
1/1 [==============================] - 0s
writing 切
1/1 [==============================] - 0s
writing 日
1/1 [==============================] - 0s
writing 家
1/1 [==============================] - 0s
writing 持
1/1 [==============================] - 0s
writing 京
1/1 [==============================] - 0s
writing 为
1/1 [==============================] - 0s
writing 文
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 有
1/1 [==============================] - 0s
writing 观
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing 外
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing 米
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 。
1/1 [=======

1/1 [==============================] - 0s
writing 要
1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 后
1/1 [==============================] - 0s
writing 、
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 生
1/1 [==============================] - 0s
writing 示
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 国
1/1 [==============================] - 0s
writing 域
1/1 [==============================] - 0s
writing 示
1/1 [==============================] - 0s
writing 染
1/1 [==============================] - 0s
writing 快
1/1 [==============================] - 0s
writing 得
1/1 [==============================] - 0s
writing 三
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 都
1/1 [==============================] - 0s
writing 宋
1/1 [=======

1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 革
1/1 [==============================] - 0s
writing 新
1/1 [==============================] - 0s
writing 在
1/1 [==============================] - 0s
writing 括
1/1 [==============================] - 0s
writing 个
1/1 [==============================] - 0s
writing 在
1/1 [==============================] - 0s
writing 等
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 是
1/1 [==============================] - 0s
writing 力
1/1 [==============================] - 0s
writing 取
1/1 [==============================] - 0s
writing 是
1/1 [==============================] - 0s
writing 。
1/1 [==============================] - 0s
writing 断
1/1 [==============================] - 0s
writing 两
1/1 [==============================] - 0s
writing 为
1/1 [==============================] - 0s
writing 次
1/1 [==============================] - 0s
writing 业
1/1 [=======

1/1 [==============================] - 0s
writing 三
1/1 [==============================] - 0s
writing 斯
1/1 [==============================] - 0s
writing 者
1/1 [==============================] - 0s
writing 队
1/1 [==============================] - 0s
writing 此
1/1 [==============================] - 0s
writing 用
1/1 [==============================] - 0s
writing 有
1/1 [==============================] - 0s
writing ８
1/1 [==============================] - 0s
writing ，
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 名
1/1 [==============================] - 0s
writing 心
1/1 [==============================] - 0s
writing 群
1/1 [==============================] - 0s
writing 的
1/1 [==============================] - 0s
writing 力
1/1 [==============================] - 0s
writing 多
1/1 [==============================] - 0s
writing 苏
1/1 [==============================] - 0s
writing 群
1/1 [==============================] - 0s
writing 别
1/1 [=======

In [33]:
out_file = open('./output_rnn', 'rb')

In [34]:
out_file.read()

'\xe6\xb3\x95 \xe5\x88\x92 \xe7\x94\xb5\xe5\x9b\xbd \xe8\xaf\xb4 \xe9\xbb\x91\xe5\xb0\x94 \xe8\xae\xb0 \xe8\xaf\x9d \xe8\xae\xae\xe8\xbf\x90\xe5\x8f\xaf\xe7\x9a\x84 \xef\xbc\x8c \xe5\xa4\xa7\xe8\x8e\xb1 \xe5\x9b\xbd \xe5\x88\xb0 \xe4\xb8\x8d\xe6\xad\xa6\xe6\x9c\xb1\xe8\xa7\xa3\xe6\x98\xaf \xe4\xbb\xa5 \xe9\x98\x9f \xe5\xb9\xb4 \xe6\x9d\xa5 \xe9\xa9\xbb\xe3\x80\x82 \xe7\x9b\x94 \xe3\x80\x82 \xef\xbc\x91\xe5\xa1\x9e \xe4\xb8\x8d\xe4\xbb\x96 \xe8\xae\xae\xe7\xa6\x81\xe9\x80\x9a\xe8\xb0\x8b\xe6\xb3\xa2\xef\xbc\x89 \xe7\x90\x86 \xe7\x94\xb5\xe5\x86\x9b \xe5\x9c\xa8 \xe5\x9b\xbd \xe7\x93\xa6 \xef\xbc\x8c \xe6\xb3\xb0 \xe4\xb9\x8b\xe5\x9b\xbd \xe5\x8d\x8e \xe6\x9c\x89 \xe5\x85\xb3 \xef\xbc\x8c \xe6\x9d\xa5 \xe9\xab\x98\xe5\x88\xa9 \xe4\xb8\x80 \xe7\x9a\x84 \xef\xbc\x8c \xe4\xb8\xbb\xe4\xb8\xad\xe5\x9b\xbd \xe9\xab\x98\xe6\xb8\x85\xe5\x9b\xbd \xe5\xbc\x8f \xe5\x8c\x88\xe9\xa6\x96\xe4\xb9\x9d\xe4\xba\xba \xe8\xa1\x8c \xef\xbc\x9a \xe9\x82\x80\xe7\xbb\x9f\xe4\xb9\x9d\xe4\xb8\xad\xe8\xbf\x98 \xe6

In [4]:
import numpy as np
def pad_sequences(sequences, maxlen=10, dtype='int32',
                  padding='pre', truncating='post', value=-1):
    
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    lengths = []
    for x in sequences:
        if not hasattr(x, '__len__'):
            raise ValueError('`sequences` must be a list of iterables. '
                             'Found non-iterable: ' + str(x))
        lengths.append(len(x))

    num_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((num_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [5]:
X = [[0,1,0,1,2], [1,2,4,0,0,1,2], [0,1,2]]
y = [[0,1],[0,0],[0,1],[0,1],[0,0]], [[0,1],[1,1],[0,0],[1,1],[0,0],[1,0],[0,1]], [[0,1],[1,1],[0,0]]

In [9]:
pad_sequences(X, maxlen=2, value=-1, truncating='pre')

array([[1, 2],
       [1, 2],
       [1, 2]], dtype=int32)

In [38]:
x = 'this is anmol sharma and ben'

y = [['this'], [' is '], ['anmo']]

splits = []
dlength = 4
# count = len(x) / 4
# for i in range(count):
#     splits.append(x[i*dlength : (i+1)*dlength])
q = x.split()

def label(word):
    wlen = len(word)
    assert(wlen)
    labels = []
    labels = [(word[0], 'B')]
    if len(word) > 1:
        for i in range(1, wlen-1):
            labels.append((word[i], 'M'))
        labels.append((word[wlen-1], 'E'))
    else:
        labels = [(word[0], 'S')]
    return labels
        
def labelsentence(wordlist):
    return [label(word) for word in wordlist]

def ngramlabel(labelledlist, n):
    # list of letter, label
    merged = []
    for l in labelledlist:
        merged += l
    count = len(merged)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( merged[i*n : (i+1)*n])
    return ngrammedlist    
    
    


In [39]:
q=labelsentence(x.split())
ngramlabel(q,2)



[[('t', 'B'), ('h', 'M')],
 [('i', 'M'), ('s', 'E')],
 [('i', 'B'), ('s', 'E')],
 [('a', 'B'), ('n', 'M')],
 [('m', 'M'), ('o', 'M')],
 [('l', 'E'), ('s', 'B')],
 [('h', 'M'), ('a', 'M')],
 [('r', 'M'), ('m', 'M')],
 [('a', 'E'), ('a', 'B')],
 [('n', 'M'), ('d', 'E')],
 [('b', 'B'), ('e', 'M')]]

In [40]:
q

[[('t', 'B'), ('h', 'M'), ('i', 'M'), ('s', 'E')],
 [('i', 'B'), ('s', 'E')],
 [('a', 'B'), ('n', 'M'), ('m', 'M'), ('o', 'M'), ('l', 'E')],
 [('s', 'B'), ('h', 'M'), ('a', 'M'), ('r', 'M'), ('m', 'M'), ('a', 'E')],
 [('a', 'B'), ('n', 'M'), ('d', 'E')],
 [('b', 'B'), ('e', 'M'), ('n', 'E')]]

In [55]:
# we open the count file, get the word and assign labels to each character
# cannot use dictionaries, since the same character may appear again and overwrites the value at its place in dict.
def generateTupleList(filename='/local-scratch/asa224/word_file_1M'):
    with(open(filename, 'rb')) as f:
        label = []
        for line in f:
            word, count = line.split('\t')

            # making sure the parsing is going fine
            assert int(count) == 0

            word = unicode(word, 'utf-8')
            if len(word) == 1:
                label.append((word[0], 3))
            else:
                for i, character in enumerate(word):
                    if i == 0: # this is the first letter
                        label.append((character, 0))
                    elif i == (len(word) - 1): # this is the last letter
                        label.append((character, 2))
                    else: # this is somewhere in the middle
                        label.append((character, 1))

        f.close()
        return label
    
def nGramSequenceGenerator(labelledlist, n):
    """
    Takes as input the label list of tuples generated by the code above. 
    The function generates sequence of size "n" from the given list. 
    """
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist   

In [56]:
label

[(u'\u7531', 0),
 (u'\u4e8e', 2),
 (u'\u5927', 0),
 (u'\u9646', 2),
 (u'\u8d74', 3),
 (u'\u53f0', 3),
 (u'\u65c5', 0),
 (u'\u6e38', 2),
 (u'\u8003', 0),
 (u'\u5bdf', 2),
 (u'\u56e2', 3),
 (u'\u7684', 3),
 (u'\u884c', 0),
 (u'\u7a0b', 1),
 (u'\u5b89', 1),
 (u'\u6392', 2),
 (u'\u5341', 0),
 (u'\u5206', 2),
 (u'\u7d27', 0),
 (u'\u51d1', 2),
 (u'\uff0c', 3),
 (u'\u56e2', 0),
 (u'\u5458', 1),
 (u'\u4eec', 2),
 (u'\u66f4', 0),
 (u'\u591a', 1),
 (u'\u7684', 2),
 (u'\u65f6', 0),
 (u'\u5019', 2),
 (u'\u53ea', 3),
 (u'\u80fd', 3),
 (u'\u662f', 3),
 (u'\u5728', 3),
 (u'\u65c5', 0),
 (u'\u884c', 2),
 (u'\u8f66', 3),
 (u'\u4e0a', 3),
 (u'\u8d70', 0),
 (u'\u9a6c', 1),
 (u'\u89c2', 1),
 (u'\u82b1', 1),
 (u'\u5730', 2),
 (u'\u8003', 0),
 (u'\u5bdf', 2),
 (u'\u53f0', 0),
 (u'\u5317', 2),
 (u'\u5e02', 3),
 (u'\u5bb9', 3),
 (u'\uff0c', 3),
 (u'\u6240', 0),
 (u'\u4ee5', 2),
 (u'\u4eba', 0),
 (u'\u6c11', 1),
 (u'\u5e01', 2),
 (u'\u4e0a', 3),
 (u'\u5468', 0),
 (u'\u7684', 2),
 (u'\u8d2c', 0),
 (u'\u503c', 2

In [87]:
def nGramSequenceGenerator(labelledlist, n):
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist   

In [88]:
nGramSequenceGenerator(label[0:10], n=2)

[[(u'\u7531', 0), (u'\u4e8e', 2)],
 [(u'\u5927', 0), (u'\u9646', 2)],
 [(u'\u8d74', 3), (u'\u53f0', 3)],
 [(u'\u65c5', 0), (u'\u6e38', 2)],
 [(u'\u8003', 0), (u'\u5bdf', 2)]]

In [89]:
nGramSequenceGenerator(label[0:12], n=3)

[[(u'\u7531', 0), (u'\u4e8e', 2), (u'\u5927', 0)],
 [(u'\u9646', 2), (u'\u8d74', 3), (u'\u53f0', 3)],
 [(u'\u65c5', 0), (u'\u6e38', 2), (u'\u8003', 0)],
 [(u'\u5bdf', 2), (u'\u56e2', 3), (u'\u7684', 3)]]